In [1]:
import pandas as pd
import numpy as np
import random
import csv
from pdb import set_trace
import pickle
from copy import deepcopy
#%pdb
mypath = ('')
trials = {}
categories = [' bottle', ' horse', ' pottedplant', ' dog',
                  ' cat', ' person', ' aeroplane', ' car', ' chair', ' sofa', ' bird', ' boat']
odd = categories[::2]
even = categories[1::2]
p_load = ['low','high']

pd.set_option('display.max_rows', 9000)
pd.set_option('display.max_columns', 9000)

In [2]:
def present_absent_trials(input_1, input_2):
    global target_present, lista, categories, p_load, odd, even
    # DEFINING TARGET-PRESENT PROBES
    
    #target_present = []
    intermediate = []
    
#next two lines are to check that the images picked as target present and target absent trials
                    #are different from the ones chosen in previous iterations
    for category in categories:
        for load in p_load:
            while True:
                trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
                if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                    and trial not in intermediate and trial not in target_present\
                    and 'Target Present' not in possible_images_dict[trial]\
                    and 'Target Absent' not in possible_images_dict[trial]:
                    intermediate.append(trial)
                if len(intermediate) == 1:
                    for each in intermediate:
                        target_present.append(each)
                        possible_images_dict[each].append(category)
                    intermediate = []
                    break

    for category in input_1:
        for load in p_load:
            while True:
                trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
                if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                    and trial not in intermediate and trial not in target_present\
                    and 'Target Present' not in possible_images_dict[trial]\
                    and 'Target Absent' not in possible_images_dict[trial]:
                    intermediate.append(trial)
                if len(intermediate) == 1:
                    for each in intermediate:
                        target_present.append(each)
                        possible_images_dict[each].append(category)
                    intermediate = []
                    break
    #print len(target_present)
    for n in target_present:
        possible_images_dict[n].append('Target Present')

    #target_present = [n for n in lista if 'Target Present' in possible_images_dict[n]]


    # DEFINING TARGET-ABSENT PROBES
    #target_absent = []
    intermediate = []
    for category in categories:
        for load in p_load:
            while True:
                trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
                if category not in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                    and trial not in intermediate and trial not in target_absent and trial not in target_present\
                    and 'Target Present' not in possible_images_dict[trial]\
                    and 'Target Absent' not in possible_images_dict[trial]:
                    intermediate.append(trial)
                if len(intermediate) == 1:
                    for each in intermediate:
                        target_absent.append(each)
                        possible_images_dict[each].append(category)
                    intermediate = []
                    break

    for category in input_2:
        for load in p_load:
            while True:
                trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
                if category not in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                    and trial not in intermediate and trial not in target_absent and trial not in target_present\
                    and 'Target Present' not in possible_images_dict[trial]\
                    and 'Target Absent' not in possible_images_dict[trial]:
                    intermediate.append(trial)
                if len(intermediate) == 1:
                    for each in intermediate:
                        target_absent.append(each)

                        possible_images_dict[each].append(category)
                    intermediate = []
                    break

    for each in target_absent:
        possible_images_dict[each].append('Target Absent')
        

    # DEFINING NO-TARGET IMAGES
    for image in lista:
        if image not in target_present and image not in target_absent:
            possible_images_dict[image].append('NA')
            possible_images_dict[image].append('No Target')
    
    return target_present, target_absent




In [3]:
# DEFINING CRITICAL TRIALS
#Criteria:
# - images of set load level
# - images containing a certain category
# - image is not already in the list of critical trials for this level of load
#- image is not aready in the list of critical trials (regardless of level of load)

def pre_select_critical_trial(load_level):
    global categories, critical_trials
    temp = []
    container = []
    for element in lista:
        if load_level in possible_images_dict[element]:
            container.append(element)
    for category in categories:
        while len(temp) <1 :
            intermediate = str(np.random.choice(container, 1, replace= False))[2:-2]
            if category in possible_images_dict[intermediate] \
                and intermediate not in temp and intermediate not in critical_trials \
                and 'Critical' not in possible_images_dict[intermediate]: # this is just a sanity check
                temp.append(intermediate)
        for each in temp:
            critical_trials.append(each)      
        temp =[]

In [4]:
def appending_critical_trials():
    global critical_low_present_probes, critical_low_absent_probes, critical_high_present_probes, critical_high_absent_probes
    global critical_trials
    cycles = 1
    while True:

        #picking critical trials
        pre_select_critical_trial('low')
        pre_select_critical_trial('high')      

        # counting critical probes
        crit_p = 0
        for n in critical_trials:
            if 'No Target' not in possible_images_dict[n]:
                crit_p = crit_p +1


        critical_low_counter = 0
        critical_high_counter = 0
        crit_p = 0

        for n in critical_trials:
            if 'No Target' not in possible_images_dict[n]:
                crit_p = crit_p +1
            if 'No Target' in possible_images_dict[n] and 'low' in possible_images_dict[n]:
                critical_low_counter = critical_low_counter + 1
            if 'No Target' in possible_images_dict[n] and 'high' in possible_images_dict[n]:
                critical_high_counter = critical_high_counter +1
            if 'Target Present' in possible_images_dict[n] and 'low' in possible_images_dict[n]:
                critical_low_present_probes.append(n)
            if 'Target Absent' in possible_images_dict[n] and 'low' in possible_images_dict[n]:
                critical_low_absent_probes.append(n)
            if 'Target Present' in possible_images_dict[n] and 'high' in possible_images_dict[n]:
                critical_high_present_probes.append(n)
            if 'Target Absent' in possible_images_dict[n] and 'high' in possible_images_dict[n]:
                critical_high_absent_probes.append(n)
        #print crit_p
        #print critical_low_counter
        #print critical_high_counter
        #print critical_low_present_probes
        #print critical_low_absent_probes
        #print critical_high_present_probes
        #print critical_high_absent_probes

        #######################################################
        if crit_p == 0 or len(critical_low_present_probes) == len(critical_high_present_probes) and len(critical_low_absent_probes) == \
        len(critical_high_absent_probes): #check if there is an even number of critical probes
            break
            print 'cycles : ',cycles
        else:
            cycles = cycles + 1
            critical_low_present_probes = []
            critical_low_absent_probes = []
            critical_high_present_probes = []
            critical_high_absent_probes = []
            #print len(critical_low_present_probes)
            #print len(critical_high_present_probes)
            #print len(critical_low_absent_probes)
            #print len(critical_high_absent_probes)
            #print '#######################'

            #why did I reset target_present and target_absent???

            #target_present = []
            #target_absent = []
            critical_trials = []

    # appendinf words 'critical' and 'normal' to dict
    for each in critical_trials:
        possible_images_dict[each].append('Critical')
    for each in possible_images_dict:
        if 'Critical' not in possible_images_dict[each]:
            possible_images_dict[each].append('Normal')

    #######################################################
    #print 'Probe trials: ', len(probe_trials_1)
    print 'Target Present Probes: ', len(target_present)
    print 'Target Absent probes: ', len(target_absent)
    print 'Critical trials:', len(critical_trials)
    print 'Critical, low: ' , critical_low_counter
    print 'Critical, high: ' , critical_high_counter
    print 'Are all critical trials unique? ', len(critical_trials) == len(set(critical_trials))
    #######################################################
    print 'Number of critical probes: ' , crit_p
    print 'critical_low_present_probes: ' , len(critical_low_present_probes)
    print 'critical_high_present_probes: ' , len(critical_high_present_probes)
    print 'critical_low_absent_probes: ' , len(critical_low_absent_probes)
    print 'critical_high_absent_probes: ', len(critical_high_absent_probes)
    print 'cycles : ',cycles
    
    return

In [5]:
def critical_probes_distribution(starting):
    global critical_low_present_probes, critical_low_absent_probes
    while True:
        p_left = len(starting)
        p_max = p_left
        distribution = []
        for i in range(4):
            if p_left > 0:
                x = random.randint(0,p_left)
                distribution.append(x)
                p_left = p_left - x
            else:
                distribution.append(0)
        if sum(distribution) == p_max:
            break
    random.shuffle(distribution)
    return distribution


In [6]:
def block_pair(present_n,absent_n):
# Pick 4 present probes and 5 absent probes
    global critical_per_block,probes_per_block, present_used, absent_used, critical_used, normal_per_block
    global present_crit_distribution, absent_crit_distribution
    global present_critical_per_block, absent_critical_per_block, critical_probes_used, critical_probes_set
    global present_per_block, absent_per_block
    
    present_crit = present_crit_distribution.pop(0)
    absent_crit = absent_crit_distribution.pop(0)
    block = []
    
    for load in p_load:
        #PRINTING STUFF
        print 'present_crit_distribution_original: ',present_crit_distribution_original
        print 'absent_crit_distribution_original: ', absent_crit_distribution_original
        print 'present_crit', present_crit
        print 'absent_crit', absent_crit
        print 'present_crit_distribution: ', present_crit_distribution
        print 'absent_crit_distribution: ', absent_crit_distribution
        print 'critical present set: ', critical_present_set
        print 'critical absent set: ', critical_absent_set
        print'---------------------------------------------------'
        normal_availble = [n for n in normal if n not in normal_used]
        print 'normal available:', len(normal_availble)
    # PICKING CRITICAL PROBES    
    #------------------------------------------------------------------------------
        if present_crit != 0:
            present_critical_per_block = []
            while len(present_critical_per_block) < present_crit:
                crit_probe = str(np.random.choice(critical_present_set,1,replace = False))[2:-2]
                if crit_probe not in present_critical_per_block and \
                crit_probe not in critical_probes_used and load in possible_images_dict[crit_probe] and \
                'Target Present' in possible_images_dict[crit_probe]:

                    present_critical_per_block.append(crit_probe)
            critical_present.append(present_critical_per_block)
                    
        print 'present_critical_per_block:', present_critical_per_block
        
        if absent_crit != 0:
            absent_critical_per_block = []
            while len(absent_critical_per_block) < absent_crit:
                crit_probe = str(np.random.choice(critical_absent_set,1,replace = False))[2:-2]
                if crit_probe not in absent_critical_per_block and \
                crit_probe not in critical_probes_used and load in possible_images_dict[crit_probe] and \
                'Target Absent' in possible_images_dict[crit_probe]:

                    absent_critical_per_block.append(crit_probe)
            critical_absent.append(absent_critical_per_block)
            
        print 'absent_critical_per_block:', absent_critical_per_block  
        print 'critical_present: ',critical_present
        print 'critical_absent: ',critical_absent
        
        for n in present_critical_per_block + absent_critical_per_block:
            critical_probes_used.append(n)
            
        print 'critical_probes_used: ', critical_probes_used
    #------------------------------------------------------------------------------
    # PICKING PRESENT AND ABSENT PROBES
        total = present_n + absent_n
        while len(probes_per_block) < (present_n - present_crit):
            probe = str(np.random.choice(target_present,1,replace = False))[2:-2]
            if probe not in probes_per_block and probe not in present_used and load in possible_images_dict[probe] \
            and 'Critical' not in possible_images_dict[probe]:
                probes_per_block.append(probe)
                
        print 'probes_per_block :', probes_per_block
        
        while (present_n - present_crit) <= len(probes_per_block) < total - present_crit - absent_crit:
            probe = str(np.random.choice(target_absent,1,replace = False))[2:-2]
            if probe not in probes_per_block and probe not in absent_used and load in possible_images_dict[probe] \
            and 'Critical' not in possible_images_dict[probe]:
                probes_per_block.append(probe)
                
        print 'probes_per_block :', probes_per_block
        
        for n in probes_per_block:
            if 'Target Present' in possible_images_dict[n]:
                present_used.append(n)
            elif 'Target Absent' in possible_images_dict[n]:
                absent_used.append(n)

        probes.append(probes_per_block)
        
        print 'probes: ', probes
        present_per_block = [n for n in probes_per_block if 'Target Present' in possible_images_dict[n]]
        absent_per_block = [n for n in probes_per_block if 'Target Absent' in possible_images_dict[n]]
        print 'Present Used in block: ',len(present_per_block)
        print 'Absent Used in block',len(absent_per_block)
    #------------------------------------------------------------------------------    
    #Counting critical probes    
        critical_probes_counter = len(present_critical_per_block + absent_critical_per_block)
        print 'Critical probes counter: ',critical_probes_counter
        print 'Present Critical: ', len(present_critical_per_block)
        print 'Absent Critical: ', len(absent_critical_per_block)

        modifier.append(critical_probes_counter)
    #------------------------------------------------------------------------------
    #Picking critical trials
        while len(critical_per_block) < (3-critical_probes_counter):
            image = str(np.random.choice(critical_trials,1,replace = False))[2:-2]
            if image not in critical_per_block and image not in critical_used \
                and 'No Target' in possible_images_dict[image] and load in possible_images_dict[image]:
                critical_per_block.append(image) 
        print 'List of critical trials without probe: '
        for n in critical_per_block:
            critical_used.append(n)
            print possible_images_dict[n]
        #temp = []

        critical.append(critical_per_block)

    #------------------------------------------------------------------------------
    #Picking normal trials
        while len(normal_per_block) < 51 - (3 - critical_probes_counter):
            image = str(np.random.choice(normal,1,replace = False))[2:-2]
            if image not in normal_per_block and image not in normal_used and load in possible_images_dict[image]:
                normal_per_block.append(image)
        for n in normal_per_block:
            normal_used.append(n)
        print 'normal in this block: ', len(normal_per_block)
        print 'normal used overall: ',len(normal_used)

        normal_builder.append(normal_per_block)
    #----------------------------------------------------------------------------
        
        print 'critical_probes_used', critical_probes_used 
        
        
        
        if 'sequence_final' in locals():
            print 'sequence_final exists'
        else:
            print 'sequence final does not exist'
            sequence_final = position_finder()
            print sequence_final
        print'---------------------------------------------------'
        print 'END OF BLOCK PROCESSING'
        print'---------------------------------------------------'
        print 'BUILDING THE BLOCK'
        first_element = [x[0] for x in sequence_final]
        second_element = [x[1] for x in sequence_final]
        second_element_counter = 0
        probes_position = []
        for image in range(60):
            #print [x[1] for x in sequence_final if x[0] ==image][0]
            if image in first_element:
                
                print 'image', image, second_element[second_element_counter]
        
                if second_element[second_element_counter] == 'Target Present':
                    probes_position.append(image)
                    while True:
                        element = str(np.random.choice(present_per_block,1,replace = False))[2:-2]
                        if element not in block:
                            block.append(element)
                            second_element_counter = second_element_counter +1
                            break       
                elif second_element[second_element_counter] == 'Target Absent':
                    probes_position.append(image)
                    while True:
                        element = str(np.random.choice(absent_per_block,1,replace = False))[2:-2]
                        if element not in block:
                            block.append(element)
                            second_element_counter = second_element_counter +1
                            break   
                elif second_element[second_element_counter] == 'Critical':
                    while True:
                        element = str(np.random.choice(critical_per_block,1,replace = False))[2:-2]
                        if element not in block:
                            block.append(element)
                            second_element_counter = second_element_counter +1
                            break     
                elif second_element[second_element_counter] == 'Critical Present':
                    probes_position.append(image)
                    while True:
                        element = str(np.random.choice(present_critical_per_block,1,replace = False))[2:-2]
                        if element not in block:
                            block.append(element)
                            second_element_counter = second_element_counter +1
                            break   
                elif second_element[second_element_counter] == 'Critical Absent':
                    probes_position.append(image)
                    while True:
                        element = str(np.random.choice(absent_critical_per_block,1,replace = False))[2:-2]
                        if element not in block:
                            block.append(element)
                            second_element_counter = second_element_counter +1
                            break  
            else:
                while True:
                    element = str(np.random.choice(normal_per_block,1,replace = False))[2:-2]
                    if element not in block:
                        block.append(element)
                        break
                
        print 'block:', len(block)   
        print'---------------------------------------------------'
        print 'probes_position: ', probes_position
    #-----------------------------------------------------------------------------
        normal_per_block = []
        probes_per_block = []
        critical_per_block = []
        present_critical_per_block = []
        absent_critical_per_block = []
    #-----------------------------------------------------------------------------
    return block

In [7]:
def position_finder():
    global present_critical_per_block, absent_critical_per_block, present_per_block, absent_per_block, critical_per_block
    x = len(present_critical_per_block + absent_critical_per_block)
    print 'x', x
    print 'present_critical_per_block', len(present_critical_per_block)
    print 'absent_critical_per_block', len(absent_critical_per_block)
    while True:    
        while True:
            a = [random.randint(1,10) for r in xrange(12-x)]
            if sum(a) <= 60:
                break

        new_list_1 = []
        acc = 0
        for i in a:
            acc = acc + i
            new_list_1.append(acc)
        new_list_1 = [n - 1 for n in new_list_1]

        if len(present_critical_per_block) == 0 and len(absent_critical_per_block) == 0:
            print 'ciao', len(critical_per_block)

            sequence = ['Target Present'] * len(present_per_block) + ['Target Absent'] * len(absent_per_block) +\
            ['Critical'] * len(critical_per_block)
            sequence = random.sample(sequence, len(sequence))

        if len(present_critical_per_block) != 0 or len(absent_critical_per_block) != 0:
            print 'ciao2', len(critical_per_block)

            sequence = ['Target Present'] * len(present_per_block)  +\
            ['Target Absent'] * len(absent_per_block) +\
            ['Critical'] * len(critical_per_block) +\
            ['Critical Present'] * len(present_critical_per_block) +\
            ['Critical Absent'] * len(absent_critical_per_block)
            sequence = random.sample(sequence, len(sequence))

        sequence_final = zip(new_list_1, sequence)
        #print sequence_final
        #print len(sequence_final)
        gap_check = [n[0] for n in sequence_final if 'Critical' in n[1]] 
        #print gap_check
        container = []
        for i in gap_check:
            elements = [i+1, i+2]
            for element in elements:
                if element in new_list_1:
                    container.append(element)
        #print 'container', len(container),container
        if len(container) == 0:
            break
        else:
            sequence_final = []

    print '------------------'
    print new_list_1, len(new_list_1)
    print 'gap_check: ', gap_check
    print len(sequence_final)

    for i in sequence_final:
        print i
    return sequence_final

In [8]:
def block_unpacker(splitting_index,present,absent):
    block = block_pair(present,absent) #, probes_position
    low_block = []
    high_block = []
    for i in range(len(block)):
        if i < (len(block)/2):
            low_block.append(block[i])
        else:
            high_block.append(block[i])
    #print probes_position
    print len(low_block), len(high_block)
    return low_block, high_block #, probes_position

In [9]:
def tone_properties():
    while True:
        possible_tones = ['tone1','tone2']
        low_one_tones = [np.random.choice(possible_tones) for r in xrange(3)]
        low_two_tones = [np.random.choice(possible_tones) for r in xrange(3)]
        low_three_tones = [np.random.choice(possible_tones) for r in xrange(3)]
        low_four_tones = [np.random.choice(possible_tones) for r in xrange(3)]
        high_one_tones = []
        high_two_tones = []
        high_three_tones = []
        high_four_tones = []
        for each in low_one_tones:
            if each == 'tone1':
                high_one_tones.append('tone2')
            elif each == 'tone2':
                high_one_tones.append('tone1')
        for each in low_two_tones:
            if each == 'tone1':
                high_two_tones.append('tone2')
            elif each == 'tone2':
                high_two_tones.append('tone1')
        for each in low_three_tones:
            if each == 'tone1':
                high_three_tones.append('tone2')
            elif each == 'tone2':
                high_three_tones.append('tone1')
        for each in low_four_tones:
            if each == 'tone1':
                high_four_tones.append('tone2')
            elif each == 'tone2':
                high_four_tones.append('tone1')
        sound_Hz = low_one_tones + low_two_tones + low_three_tones + low_four_tones + high_one_tones + high_two_tones +\
        high_three_tones + high_four_tones

        tone1_counter = 0
        tone2_counter = 0
        for each in sound_Hz:
            if each == 'tone1':
                tone1_counter = tone1_counter + 1
            elif each == 'tone2':
                tone2_counter = tone2_counter + 1

        low_load_tones = [n for n in sound_Hz[0:12]]
        high_load_tones = [n for n in sound_Hz[12:25]]
        low_load_tone1s = [n for n in low_load_tones if n == 'tone1']
        low_load_tone2s = [n for n in low_load_tones if n == 'tone2']
        high_load_tone1s = [n for n in high_load_tones if n == 'tone1']
        high_load_tone2s = [n for n in high_load_tones if n =='tone2']

        if len(low_load_tone1s) == len(low_load_tone2s) and len(high_load_tone1s) == len(high_load_tone2s):
            print low_one_tones
            print high_one_tones
            print '----------------------'
            print low_two_tones
            print high_two_tones
            print '----------------------'
            print low_three_tones
            print high_three_tones
            print '----------------------'
            print low_four_tones
            print high_four_tones
            print '----------------------'
            print 'total n of tones: ', len(sound_Hz)
            print '----------------------'
            print 'total n of 1500 Hz tones: ', tone1_counter
            print 'total n of 2000 Hz tones: ', tone2_counter
            print '----------------------'
            print len(low_load_tone1s) == len(low_load_tone2s)
            print len(low_load_tone1s)
            print len(low_load_tone2s)
            print len(high_load_tone1s) == len(high_load_tone2s)
            print len(high_load_tone1s)
            print len(high_load_tone2s)
            break
        # Re-ordering tones according to the block order
    sound_Hz_new = low_one_tones + high_two_tones + high_one_tones + low_two_tones + high_three_tones + low_four_tones +\
        low_three_tones + high_four_tones 
    print sound_Hz_new
    sound_onset = []
    for i in range(48):
        a = random.uniform(0, 0.999)
        a = int(str(a)[2:5])
        #print a
        sound_onset.append(a)

    #------------------------------------------------------------------------------
    # APPEND TO DICT
    sound_index = 0
    for each in possible_images_dict:
        if 'Critical' in possible_images_dict[each]:
            possible_images_dict[each].append(sound_Hz_new[sound_index])
            print sound_Hz_new[sound_index]
            possible_images_dict[each].append(sound_onset[sound_index])
            sound_index = sound_index + 1
        elif 'Normal' in possible_images_dict[each]:
            possible_images_dict[each].append('No tone')
            possible_images_dict[each].append(-999)

In [10]:
def appending_to_dict(trial_n):
    global blocks
    for b in range(8):
        this_block_images = blocks[b]
        for im in this_block_images:
            trials[trial_n] = possible_images_dict[im]
            trials[trial_n].append(im)
            trial_n += 1
    print 'len(trials): ', len(trials)

In [11]:
######################
### FIRST ITERATION
######################

#GLOBALS

############
critical_low_present_probes = []
critical_low_absent_probes = []
critical_high_present_probes = []
critical_high_absent_probes = []
############
target_present = []
target_absent = []
critical_trials = []
############
df = pd.DataFrame.from_csv(mypath+'all_images_2.csv')
df.reset_index(drop=False)
image_names = df.index.tolist()
cat1 = df['0'].tolist()
cat2 = df['1'].tolist()
cat3 = df['2'].tolist()
cat4 = df['3'].tolist()
cat5 = df['4'].tolist()
ts = df['5'].tolist()
load = df['6'].tolist()
exemplar = df['7'].tolist()
possible_images_dict ={z[0]:list(z[1:]) for z in zip(image_names,cat1,cat2,cat3,cat4,cat5,ts,load,exemplar)}
lista = possible_images_dict.keys()
############

target_present, target_absent= present_absent_trials(odd, even)
appending_critical_trials()
#-----------------------------------------------------
normal_no_probe_before = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
target_present_before = deepcopy(target_present)
target_absent_before = deepcopy(target_absent)
critical_trials_before = deepcopy(critical_trials)
critical_low_present_probes_before = deepcopy(critical_low_present_probes)
critical_low_absent_probes_before = deepcopy(critical_low_absent_probes)
critical_high_present_probes_before = deepcopy(critical_high_present_probes)
critical_high_absent_probes_before = deepcopy(critical_high_absent_probes)
#-----------------------------------------------------

present_crit_distribution = critical_probes_distribution(critical_low_present_probes)
absent_crit_distribution = critical_probes_distribution(critical_low_absent_probes)

present_crit_distribution_original = deepcopy(present_crit_distribution)
absent_crit_distribution_original = deepcopy(absent_crit_distribution)
#-----------------------------------------------------

#Defininfg blocks 
normal_per_block = []
probes_per_block = []
present_per_block = []
absent_per_block = []
normal_probes_per_block = []
critical_per_block = []
present_critical_per_block = []
absent_critical_per_block = []

###############################
normal_used = []
present_used = []
absent_used = []
critical_used = []
critical_probes_used = []
normal = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
###############################
block = []
critical_probes_counter = 0 #useless???
critical_probes_set = critical_low_present_probes + critical_low_absent_probes + critical_high_present_probes + critical_high_absent_probes
critical_present_set = [n for n in critical_probes_set if 'Target Present' in possible_images_dict[n]]
critical_absent_set = [n for n in critical_probes_set if 'Target Absent' in possible_images_dict[n]]
###############################
normal_builder = []
probes = []
critical = []
critical_present = []
critical_absent = []
###############################
modifier = []
low_one, high_two = block_unpacker(60,4,5) 
low_two, high_one = block_unpacker(60,5,4) 
low_four, high_three = block_unpacker(60,4,5)
low_three, high_four = block_unpacker(60,5,4) 

tone_properties()

blocks = low_one , high_one , high_two , low_two , high_three , low_three , low_four , high_four
    
appending_to_dict(1)



Target Present Probes:  36
Target Absent probes:  36
Critical trials: 24
Critical, low:  11
Critical, high:  11
Are all critical trials unique?  True
Number of critical probes:  2
critical_low_present_probes:  0
critical_high_present_probes:  0
critical_low_absent_probes:  1
critical_high_absent_probes:  1
cycles :  13
present_crit_distribution_original:  [0, 0, 0, 0]
absent_crit_distribution_original:  [0, 0, 1, 0]
present_crit 0
absent_crit 0
present_crit_distribution:  [0, 0, 0]
absent_crit_distribution:  [0, 1, 0]
critical present set:  []
critical absent set:  ['2008_006564.jpg', '2008_007913.jpg']
---------------------------------------------------
normal available: 386
present_critical_per_block: []
absent_critical_per_block: []
critical_present:  []
critical_absent:  []
critical_probes_used:  []
probes_per_block : ['2009_004779.jpg', '2008_000760.jpg', '2009_003686.jpg', '2011_002911.jpg']
probes_per_block : ['2009_004779.jpg', '2008_000760.jpg', '2009_003686.jpg', '2011_002911

present_crit 0
absent_crit 1
present_crit_distribution:  [0]
absent_crit_distribution:  [0]
critical present set:  []
critical absent set:  ['2008_006564.jpg', '2008_007913.jpg']
---------------------------------------------------
normal available: 194
present_critical_per_block: []
absent_critical_per_block: ['2008_006564.jpg']
critical_present:  []
critical_absent:  [['2008_006564.jpg']]
critical_probes_used:  ['2008_006564.jpg']
probes_per_block : ['2008_002145.jpg', '2011_001613.jpg', '2010_004791.jpg', '2008_000734.jpg']
probes_per_block : ['2008_002145.jpg', '2011_001613.jpg', '2010_004791.jpg', '2008_000734.jpg', '2008_005735.jpg', '2010_004662.jpg', '2011_002582.jpg', '2008_006452.jpg']
probes:  [['2009_004779.jpg', '2008_000760.jpg', '2009_003686.jpg', '2011_002911.jpg', '2008_007704.jpg', '2009_002137.jpg', '2010_001993.jpg', '2008_003709.jpg', '2009_004129.jpg'], ['2009_001321.jpg', '2008_006912.jpg', '2010_005170.jpg', '2010_000792.jpg', '2009_001345.jpg', '2008_005519.jpg'

normal in this block:  48
normal used overall:  338
critical_probes_used ['2008_006564.jpg', '2008_007913.jpg']
sequence final does not exist
x 0
present_critical_per_block 0
absent_critical_per_block 0
ciao 3
------------------
[4, 7, 9, 16, 17, 25, 28, 36, 40, 46, 48, 52] 12
gap_check:  [17, 25, 48]
12
(4, 'Target Present')
(7, 'Target Absent')
(9, 'Target Present')
(16, 'Target Absent')
(17, 'Critical')
(25, 'Critical')
(28, 'Target Absent')
(36, 'Target Present')
(40, 'Target Present')
(46, 'Target Absent')
(48, 'Critical')
(52, 'Target Present')
[(4, 'Target Present'), (7, 'Target Absent'), (9, 'Target Present'), (16, 'Target Absent'), (17, 'Critical'), (25, 'Critical'), (28, 'Target Absent'), (36, 'Target Present'), (40, 'Target Present'), (46, 'Target Absent'), (48, 'Critical'), (52, 'Target Present')]
---------------------------------------------------
END OF BLOCK PROCESSING
---------------------------------------------------
BUILDING THE BLOCK
image 4 Target Present
image 7 T

In [12]:
######################
### SECOND ITERATION
######################


#GLOBALS

############
critical_low_present_probes = []
critical_low_absent_probes = []
critical_high_present_probes = []
critical_high_absent_probes = []
############
target_present = []
target_absent = []
critical_trials = []
############
df = pd.DataFrame.from_csv(mypath+'all_images_2.csv')
df.reset_index(drop=False)
image_names = df.index.tolist()
cat1 = df['0'].tolist()
cat2 = df['1'].tolist()
cat3 = df['2'].tolist()
cat4 = df['3'].tolist()
cat5 = df['4'].tolist()
ts = df['5'].tolist()
load = df['6'].tolist()
exemplar = df['7'].tolist()
possible_images_dict ={z[0]:list(z[1:]) for z in zip(image_names,cat1,cat2,cat3,cat4,cat5,ts,load,exemplar)}
lista = possible_images_dict.keys()
############

target_present, target_absent= present_absent_trials(even, odd)
appending_critical_trials()
#-----------------------------------------------------
normal_no_probe_before = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
target_present_before = deepcopy(target_present)
target_absent_before = deepcopy(target_absent)
critical_trials_before = deepcopy(critical_trials)
critical_low_present_probes_before = deepcopy(critical_low_present_probes)
critical_low_absent_probes_before = deepcopy(critical_low_absent_probes)
critical_high_present_probes_before = deepcopy(critical_high_present_probes)
critical_high_absent_probes_before = deepcopy(critical_high_absent_probes)
#-----------------------------------------------------

present_crit_distribution = critical_probes_distribution(critical_low_present_probes)
absent_crit_distribution = critical_probes_distribution(critical_low_absent_probes)

present_crit_distribution_original = deepcopy(present_crit_distribution)
absent_crit_distribution_original = deepcopy(absent_crit_distribution)
#-----------------------------------------------------

#Defininfg blocks 
normal_per_block = []
probes_per_block = []
present_per_block = []
absent_per_block = []
normal_probes_per_block = []
critical_per_block = []
present_critical_per_block = []
absent_critical_per_block = []

###############################
normal_used = []
present_used = []
absent_used = []
critical_used = []
critical_probes_used = []
normal = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
###############################
block = []
critical_probes_counter = 0 #useless???
critical_probes_set = critical_low_present_probes + critical_low_absent_probes + critical_high_present_probes + critical_high_absent_probes
critical_present_set = [n for n in critical_probes_set if 'Target Present' in possible_images_dict[n]]
critical_absent_set = [n for n in critical_probes_set if 'Target Absent' in possible_images_dict[n]]
###############################
normal_builder = []
probes = []
critical = []
critical_present = []
critical_absent = []
###############################
modifier = []
low_one, high_two = block_unpacker(60,4,5) 
low_two, high_one = block_unpacker(60,5,4) 
low_four, high_three = block_unpacker(60,4,5)
low_three, high_four = block_unpacker(60,5,4) 

tone_properties()

blocks = low_one , high_one , high_two , low_two , high_three , low_three , low_four , high_four
    
appending_to_dict(481)



Target Present Probes:  36
Target Absent probes:  36
Critical trials: 24
Critical, low:  10
Critical, high:  10
Are all critical trials unique?  True
Number of critical probes:  4
critical_low_present_probes:  0
critical_high_present_probes:  0
critical_low_absent_probes:  2
critical_high_absent_probes:  2
cycles :  5
present_crit_distribution_original:  [0, 0, 0, 0]
absent_crit_distribution_original:  [0, 2, 0, 0]
present_crit 0
absent_crit 0
present_crit_distribution:  [0, 0, 0]
absent_crit_distribution:  [2, 0, 0]
critical present set:  []
critical absent set:  ['2008_000112.jpg', '2011_002582.jpg', '2008_007378.jpg', '2011_002260.jpg']
---------------------------------------------------
normal available: 388
present_critical_per_block: []
absent_critical_per_block: []
critical_present:  []
critical_absent:  []
critical_probes_used:  []
probes_per_block : ['2008_006470.jpg', '2009_004888.jpg', '2008_000123.jpg', '2008_002372.jpg']
probes_per_block : ['2008_006470.jpg', '2009_004888.

image 8 Target Present
image 10 Critical
image 19 Target Absent
image 26 Critical Absent
image 36 Target Absent
image 37 Target Present
image 41 Target Present
image 42 Target Present
image 47 Target Present
image 57 Critical Absent
block: 120
---------------------------------------------------
probes_position:  [8, 19, 26, 36, 37, 41, 42, 47, 57]
60 60
present_crit_distribution_original:  [0, 0, 0, 0]
absent_crit_distribution_original:  [0, 2, 0, 0]
present_crit 0
absent_crit 0
present_crit_distribution:  [0]
absent_crit_distribution:  [0]
critical present set:  []
critical absent set:  ['2008_000112.jpg', '2011_002582.jpg', '2008_007378.jpg', '2011_002260.jpg']
---------------------------------------------------
normal available: 192
present_critical_per_block: []
absent_critical_per_block: []
critical_present:  []
critical_absent:  [['2008_000112.jpg', '2011_002582.jpg'], ['2008_007378.jpg', '2011_002260.jpg']]
critical_probes_used:  ['2008_000112.jpg', '2011_002582.jpg', '2008_0073

normal in this block:  48
normal used overall:  340
critical_probes_used ['2008_000112.jpg', '2011_002582.jpg', '2008_007378.jpg', '2011_002260.jpg']
sequence final does not exist
x 0
present_critical_per_block 0
absent_critical_per_block 0
ciao 3
ciao 3
ciao 3
ciao 3
ciao 3
------------------
[2, 3, 13, 15, 22, 25, 32, 34, 39, 46, 51, 54] 12
gap_check:  [15, 34, 46]
12
(2, 'Target Present')
(3, 'Target Present')
(13, 'Target Present')
(15, 'Critical')
(22, 'Target Absent')
(25, 'Target Present')
(32, 'Target Absent')
(34, 'Critical')
(39, 'Target Absent')
(46, 'Critical')
(51, 'Target Present')
(54, 'Target Absent')
[(2, 'Target Present'), (3, 'Target Present'), (13, 'Target Present'), (15, 'Critical'), (22, 'Target Absent'), (25, 'Target Present'), (32, 'Target Absent'), (34, 'Critical'), (39, 'Target Absent'), (46, 'Critical'), (51, 'Target Present'), (54, 'Target Absent')]
---------------------------------------------------
END OF BLOCK PROCESSING
----------------------------------

In [13]:
######################
### THIRD ITERATION
######################

#GLOBALS

############
critical_low_present_probes = []
critical_low_absent_probes = []
critical_high_present_probes = []
critical_high_absent_probes = []
############
target_present = []
target_absent = []
critical_trials = []
############
df = pd.DataFrame.from_csv(mypath+'all_images_2.csv')
df.reset_index(drop=False)
image_names = df.index.tolist()
cat1 = df['0'].tolist()
cat2 = df['1'].tolist()
cat3 = df['2'].tolist()
cat4 = df['3'].tolist()
cat5 = df['4'].tolist()
ts = df['5'].tolist()
load = df['6'].tolist()
exemplar = df['7'].tolist()
possible_images_dict ={z[0]:list(z[1:]) for z in zip(image_names,cat1,cat2,cat3,cat4,cat5,ts,load,exemplar)}
lista = possible_images_dict.keys()
############

target_present, target_absent= present_absent_trials(odd, even)
appending_critical_trials()
#-----------------------------------------------------
normal_no_probe_before = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
target_present_before = deepcopy(target_present)
target_absent_before = deepcopy(target_absent)
critical_trials_before = deepcopy(critical_trials)
critical_low_present_probes_before = deepcopy(critical_low_present_probes)
critical_low_absent_probes_before = deepcopy(critical_low_absent_probes)
critical_high_present_probes_before = deepcopy(critical_high_present_probes)
critical_high_absent_probes_before = deepcopy(critical_high_absent_probes)
#-----------------------------------------------------

present_crit_distribution = critical_probes_distribution(critical_low_present_probes)
absent_crit_distribution = critical_probes_distribution(critical_low_absent_probes)

present_crit_distribution_original = deepcopy(present_crit_distribution)
absent_crit_distribution_original = deepcopy(absent_crit_distribution)
#-----------------------------------------------------

#Defininfg blocks 
normal_per_block = []
probes_per_block = []
present_per_block = []
absent_per_block = []
normal_probes_per_block = []
critical_per_block = []
present_critical_per_block = []
absent_critical_per_block = []

###############################
normal_used = []
present_used = []
absent_used = []
critical_used = []
critical_probes_used = []
normal = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
###############################
block = []
critical_probes_counter = 0 #useless???
critical_probes_set = critical_low_present_probes + critical_low_absent_probes + critical_high_present_probes + critical_high_absent_probes
critical_present_set = [n for n in critical_probes_set if 'Target Present' in possible_images_dict[n]]
critical_absent_set = [n for n in critical_probes_set if 'Target Absent' in possible_images_dict[n]]
###############################
normal_builder = []
probes = []
critical = []
critical_present = []
critical_absent = []
###############################
modifier = []
low_one, high_two = block_unpacker(60,4,5) 
low_two, high_one = block_unpacker(60,5,4) 
low_four, high_three = block_unpacker(60,4,5)
low_three, high_four = block_unpacker(60,5,4) 

tone_properties()

blocks = low_one , high_one , high_two , low_two , high_three , low_three , low_four , high_four
    
appending_to_dict(961)



Target Present Probes:  36
Target Absent probes:  36
Critical trials: 24
Critical, low:  11
Critical, high:  11
Are all critical trials unique?  True
Number of critical probes:  2
critical_low_present_probes:  0
critical_high_present_probes:  0
critical_low_absent_probes:  1
critical_high_absent_probes:  1
cycles :  4
present_crit_distribution_original:  [0, 0, 0, 0]
absent_crit_distribution_original:  [0, 0, 1, 0]
present_crit 0
absent_crit 0
present_crit_distribution:  [0, 0, 0]
absent_crit_distribution:  [0, 1, 0]
critical present set:  []
critical absent set:  ['2011_001005.jpg', '2008_000541.jpg']
---------------------------------------------------
normal available: 386
present_critical_per_block: []
absent_critical_per_block: []
critical_present:  []
critical_absent:  []
critical_probes_used:  []
probes_per_block : ['2008_003170.jpg', '2009_000059.jpg', '2008_002899.jpg', '2011_000224.jpg']
probes_per_block : ['2008_003170.jpg', '2009_000059.jpg', '2008_002899.jpg', '2011_000224.

present_crit_distribution_original:  [0, 0, 0, 0]
absent_crit_distribution_original:  [0, 0, 1, 0]
present_crit 0
absent_crit 1
present_crit_distribution:  [0]
absent_crit_distribution:  [0]
critical present set:  []
critical absent set:  ['2011_001005.jpg', '2008_000541.jpg']
---------------------------------------------------
normal available: 194
present_critical_per_block: []
absent_critical_per_block: ['2011_001005.jpg']
critical_present:  []
critical_absent:  [['2011_001005.jpg']]
critical_probes_used:  ['2011_001005.jpg']
probes_per_block : ['2008_005345.jpg', '2010_002379.jpg', '2010_003746.jpg', '2009_002594.jpg']
probes_per_block : ['2008_005345.jpg', '2010_002379.jpg', '2010_003746.jpg', '2009_002594.jpg', '2008_007096.jpg', '2008_000972.jpg', '2008_006072.jpg', '2010_005257.jpg']
probes:  [['2008_003170.jpg', '2009_000059.jpg', '2008_002899.jpg', '2011_000224.jpg', '2010_004753.jpg', '2009_000260.jpg', '2011_001967.jpg', '2011_000293.jpg', '2009_001470.jpg'], ['2008_000192.

normal in this block:  48
normal used overall:  338
critical_probes_used ['2011_001005.jpg', '2008_000541.jpg']
sequence final does not exist
x 0
present_critical_per_block 0
absent_critical_per_block 0
ciao 3
ciao 3
ciao 3
------------------
[1, 4, 7, 17, 20, 21, 31, 40, 46, 49, 55, 58] 12
gap_check:  [40, 49, 55]
12
(1, 'Target Present')
(4, 'Target Absent')
(7, 'Target Absent')
(17, 'Target Absent')
(20, 'Target Present')
(21, 'Target Present')
(31, 'Target Absent')
(40, 'Critical')
(46, 'Target Present')
(49, 'Critical')
(55, 'Critical')
(58, 'Target Present')
[(1, 'Target Present'), (4, 'Target Absent'), (7, 'Target Absent'), (17, 'Target Absent'), (20, 'Target Present'), (21, 'Target Present'), (31, 'Target Absent'), (40, 'Critical'), (46, 'Target Present'), (49, 'Critical'), (55, 'Critical'), (58, 'Target Present')]
---------------------------------------------------
END OF BLOCK PROCESSING
---------------------------------------------------
BUILDING THE BLOCK
image 1 Target Pre

In [14]:
######################
### FOURTH ITERATION
######################


#GLOBALS

############
critical_low_present_probes = []
critical_low_absent_probes = []
critical_high_present_probes = []
critical_high_absent_probes = []
############
target_present = []
target_absent = []
critical_trials = []
############
df = pd.DataFrame.from_csv(mypath+'all_images_2.csv')
df.reset_index(drop=False)
image_names = df.index.tolist()
cat1 = df['0'].tolist()
cat2 = df['1'].tolist()
cat3 = df['2'].tolist()
cat4 = df['3'].tolist()
cat5 = df['4'].tolist()
ts = df['5'].tolist()
load = df['6'].tolist()
exemplar = df['7'].tolist()
possible_images_dict ={z[0]:list(z[1:]) for z in zip(image_names,cat1,cat2,cat3,cat4,cat5,ts,load,exemplar)}
lista = possible_images_dict.keys()
############

target_present, target_absent= present_absent_trials(even, odd)
appending_critical_trials()
#-----------------------------------------------------
normal_no_probe_before = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
target_present_before = deepcopy(target_present)
target_absent_before = deepcopy(target_absent)
critical_trials_before = deepcopy(critical_trials)
critical_low_present_probes_before = deepcopy(critical_low_present_probes)
critical_low_absent_probes_before = deepcopy(critical_low_absent_probes)
critical_high_present_probes_before = deepcopy(critical_high_present_probes)
critical_high_absent_probes_before = deepcopy(critical_high_absent_probes)
#-----------------------------------------------------

present_crit_distribution = critical_probes_distribution(critical_low_present_probes)
absent_crit_distribution = critical_probes_distribution(critical_low_absent_probes)

present_crit_distribution_original = deepcopy(present_crit_distribution)
absent_crit_distribution_original = deepcopy(absent_crit_distribution)
#-----------------------------------------------------

#Defininfg blocks 
normal_per_block = []
probes_per_block = []
present_per_block = []
absent_per_block = []
normal_probes_per_block = []
critical_per_block = []
present_critical_per_block = []
absent_critical_per_block = []

###############################
normal_used = []
present_used = []
absent_used = []
critical_used = []
critical_probes_used = []
normal = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
###############################
block = []
critical_probes_counter = 0 #useless???
critical_probes_set = critical_low_present_probes + critical_low_absent_probes + critical_high_present_probes + critical_high_absent_probes
critical_present_set = [n for n in critical_probes_set if 'Target Present' in possible_images_dict[n]]
critical_absent_set = [n for n in critical_probes_set if 'Target Absent' in possible_images_dict[n]]
###############################
normal_builder = []
probes = []
critical = []
critical_present = []
critical_absent = []
###############################
modifier = []
low_one, high_two = block_unpacker(60,4,5) 
low_two, high_one = block_unpacker(60,5,4) 
low_four, high_three = block_unpacker(60,4,5)
low_three, high_four = block_unpacker(60,5,4) 

tone_properties()

blocks = low_one , high_one , high_two , low_two , high_three , low_three , low_four , high_four
    
appending_to_dict(1441)



Target Present Probes:  36
Target Absent probes:  36
Critical trials: 24
Critical, low:  8
Critical, high:  8
Are all critical trials unique?  True
Number of critical probes:  8
critical_low_present_probes:  1
critical_high_present_probes:  1
critical_low_absent_probes:  3
critical_high_absent_probes:  3
cycles :  6
present_crit_distribution_original:  [0, 0, 0, 1]
absent_crit_distribution_original:  [0, 0, 1, 2]
present_crit 0
absent_crit 0
present_crit_distribution:  [0, 0, 1]
absent_crit_distribution:  [0, 1, 2]
critical present set:  ['2010_001908.jpg', '2010_006033.jpg']
critical absent set:  ['2008_001542.jpg', '2008_002899.jpg', '2009_001774.jpg', '2008_001333.jpg', '2008_005702.jpg', '2011_001221.jpg']
---------------------------------------------------
normal available: 392
present_critical_per_block: []
absent_critical_per_block: []
critical_present:  []
critical_absent:  []
critical_probes_used:  []
probes_per_block : ['2011_000224.jpg', '2008_008083.jpg', '2009_000260.jpg',

image 55 Target Absent
image 59 Critical
block: 120
---------------------------------------------------
probes_position:  [0, 4, 13, 21, 30, 33, 40, 45, 55]
60 60
present_crit_distribution_original:  [0, 0, 0, 1]
absent_crit_distribution_original:  [0, 0, 1, 2]
present_crit 0
absent_crit 1
present_crit_distribution:  [1]
absent_crit_distribution:  [2]
critical present set:  ['2010_001908.jpg', '2010_006033.jpg']
critical absent set:  ['2008_001542.jpg', '2008_002899.jpg', '2009_001774.jpg', '2008_001333.jpg', '2008_005702.jpg', '2011_001221.jpg']
---------------------------------------------------
normal available: 200
present_critical_per_block: []
absent_critical_per_block: ['2008_001542.jpg']
critical_present:  []
critical_absent:  [['2008_001542.jpg']]
critical_probes_used:  ['2008_001542.jpg']
probes_per_block : ['2010_003746.jpg', '2010_005785.jpg', '2008_000177.jpg', '2009_002374.jpg']
probes_per_block : ['2010_003746.jpg', '2010_005785.jpg', '2008_000177.jpg', '2009_002374.jpg'

normal in this block:  51
normal used overall:  341
critical_probes_used ['2008_001542.jpg', '2011_001221.jpg', '2010_001908.jpg', '2009_001774.jpg', '2008_002899.jpg']
sequence final does not exist
x 3
present_critical_per_block 1
absent_critical_per_block 2
ciao2 0
ciao2 0
------------------
[0, 5, 10, 11, 15, 23, 27, 31, 38] 9
gap_check:  [0, 15, 31]
9
(0, 'Critical Absent')
(5, 'Target Present')
(10, 'Target Absent')
(11, 'Target Present')
(15, 'Critical Present')
(23, 'Target Absent')
(27, 'Target Present')
(31, 'Critical Absent')
(38, 'Target Present')
[(0, 'Critical Absent'), (5, 'Target Present'), (10, 'Target Absent'), (11, 'Target Present'), (15, 'Critical Present'), (23, 'Target Absent'), (27, 'Target Present'), (31, 'Critical Absent'), (38, 'Target Present')]
---------------------------------------------------
END OF BLOCK PROCESSING
---------------------------------------------------
BUILDING THE BLOCK
image 0 Critical Absent
image 5 Target Present
image 10 Target Absent
i

In [15]:

print len(normal_no_probe_before)
print len(normal_used)
print len(normal_no_probe_before) == len(set(normal_used))
print len(target_present_before)
print len(present_used)
print len(target_present_before) == len(set(present_used))
print len(target_absent_before)
print len(absent_used)
print len(target_absent_before) == len(set(absent_used))

#the following is not a good check
print len(set(critical_trials_before)) == len(set(critical_used + critical_low_present_probes + critical_low_absent_probes +
                                             critical_high_present_probes + critical_high_absent_probes))

print set(critical_trials_before) == set(critical_used + critical_low_present_probes + critical_low_absent_probes +
                                             critical_high_present_probes + critical_high_absent_probes)

print len(low_four)
print len(low_one) == len(low_two) == len(low_three) == len(low_four) == len(high_one) == len(high_two) == len(high_three) == len(high_four)
print len(set(low_one) & set(low_two) & set(low_three) & set(low_four) & set(high_one) & set(high_two) & set(high_three) & set(high_four))


392
392
True
36
34
False
36
30
False
True
True
60
True
0


In [16]:
for i in range(len(trials.keys())):
    trials[i+1].append(i/60+1)

import pickle
with open('../config/randomization.pik', 'wb') as of:
    pickle.dump(trials,of)

In [17]:
df = pd.DataFrame.from_dict(trials)
df= df.T
df.columns = ['Cat.1','Cat.2','Cat.3','Cat.4','Cat.5','TS','Load','exemplar','question','present_absent','trial_type','tone_Hz','tone_onset','image_name','block']
df.TS = df.TS.astype('float')
df.to_csv(mypath + '../Config/randomization_1.csv')

In [18]:
df[df['trial_type']=='Critical']


,Cat.1,Cat.2,Cat.3,Cat.4,Cat.5,TS,Load,exemplar,question,present_absent,trial_type,tone_Hz,tone_onset,image_name,block
2,chair,dog,diningtable,NaN,NaN,18.206264,low,chair,NA,No Target,Critical,tone1,991,2008_006837.jpg,1
29,person,boat,NaN,NaN,NaN,17.997748,low,boat,NA,No Target,Critical,tone2,5,2010_002498.jpg,1
41,cat,NaN,NaN,NaN,NaN,20.606086,low,cat,NA,No Target,Critical,tone2,795,2008_007842.jpg,1
63,car,aeroplane,NaN,NaN,NaN,35.872179,high,aeroplane,NA,No Target,Critical,tone2,449,2008_000251.jpg,2
72,sofa,chair,diningtable,NaN,NaN,30.346694,high,chair,NA,No Target,Critical,tone1,965,2008_008470.jpg,2
110,sofa,bottle,pottedplant,tvmonitor,chair,33.937776,high,pottedplant,NA,No Target,Critical,tone2,860,2011_002814.jpg,2
122,cat,NaN,NaN,NaN,NaN,29.246976,high,cat,NA,No Target,Critical,tone2,15,2008_001210.jpg,3
149,aeroplane,NaN,NaN,NaN,NaN,31.142309,high,aeroplane,NA,No Target,Critical,tone1,904,2008_007629.jpg,3
161,pottedplant,person,chair,motorbike,boat,31.580490,high,boat,NA,No Target,Critical,tone1,845,2009_003351.jpg,3
183,horse,NaN,NaN,NaN,NaN,19.207956,low,horse,NA,No Target,Critical,tone1,240,2008_008533.jpg,4


In [19]:
import pickle

trial_file = 'C:/Users/nilli lab/Desktop/load/config/randomization.pik'
#trial_file = 'config/test_1.pik'

converted = False

with open(trial_file, 'r') as of:
    trials_raw = pickle.load(of)

trials = {}

cols = ['cat_1','cat_2','cat_3','cat_4','cat_5','TS','load','exemplar','question','present_absent','trial_type','tone_hz','tone_onset','image_name','block']

if converted:
    trials = trials_raw
else:
    for k in trials_raw.keys():
        l = trials_raw[k]
        trial_dict = {}
        for i in range(len(l)):
            trial_dict[cols[i]] = l[i]
        trials[k] = trial_dict

trials

{1: {'TS': 9.156390286160866,
  'block': 1,
  'cat_1': ' car',
  'cat_2': nan,
  'cat_3': nan,
  'cat_4': nan,
  'cat_5': nan,
  'exemplar': ' car',
  'image_name': '2011_000293.jpg',
  'load': 'low',
  'present_absent': 'No Target',
  'question': 'NA',
  'tone_hz': 'No tone',
  'tone_onset': -999,
  'trial_type': 'Normal'},
 2: {'TS': 18.206263900870965,
  'block': 1,
  'cat_1': ' chair',
  'cat_2': ' dog',
  'cat_3': ' diningtable',
  'cat_4': nan,
  'cat_5': nan,
  'exemplar': ' chair',
  'image_name': '2008_006837.jpg',
  'load': 'low',
  'present_absent': 'No Target',
  'question': 'NA',
  'tone_hz': 'tone1',
  'tone_onset': 991,
  'trial_type': 'Critical'},
 3: {'TS': 16.811685000593723,
  'block': 1,
  'cat_1': ' car',
  'cat_2': ' aeroplane',
  'cat_3': nan,
  'cat_4': nan,
  'cat_5': nan,
  'exemplar': ' aeroplane',
  'image_name': '2009_005036.jpg',
  'load': 'low',
  'present_absent': 'No Target',
  'question': 'NA',
  'tone_hz': 'No tone',
  'tone_onset': -999,
  'trial_typ